In [24]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

Load data and subtract water

In [25]:
data_path = Path('~/workspace/data/lipid/GIXD').expanduser()
plot_path = Path('~/workspace/plot/lipid/GIXD').expanduser()

In [26]:
water_path = data_path / 'water'
water_index = [44]
water_pressure = [0.4]
water_I_path = [water_path / f'water_{i}_{i}_combined_I.dat' for i in water_index]
water_Qxy_path = [water_path / f'water_{i}_{i}_combined_Qxy.dat' for i in water_index]
water_Qz_path = [water_path / f'water_{i}_{i}_combined_Qz.dat' for i in water_index]
water_I = [np.loadtxt(path) for path in water_I_path]
water_Qxy = [np.loadtxt(path) for path in water_Qxy_path]
water_Qz = [np.loadtxt(path) for path in water_Qz_path]

In [27]:
azo_cis_1_path = data_path / 'azo_cis_1'
azo_cis_1_index = [78, 82, 86, 90, 94]
azo_cis_1_pressure = [5, 10, 20, 30, 30]
azo_cis_1_I_path = [azo_cis_1_path / f'azocis_{i}_{i}_combined_I.dat' for i in azo_cis_1_index]
azo_cis_1_Qxy_path = [azo_cis_1_path / f'azocis_{i}_{i}_combined_Qxy.dat' for i in azo_cis_1_index]
azo_cis_1_Qz_path = [azo_cis_1_path / f'azocis_{i}_{i}_combined_Qz.dat' for i in azo_cis_1_index]
azo_cis_1_I = [np.loadtxt(path) for path in azo_cis_1_I_path]
azo_cis_1_Qxy = [np.loadtxt(path) for path in azo_cis_1_Qxy_path]
azo_cis_1_Qz = [np.loadtxt(path) for path in azo_cis_1_Qz_path]
azo_cis_1_I -= water_I[0]
azo_cis_1_save_path = plot_path / 'azo_cis_1'
azo_cis_1_save_path.mkdir(parents=True, exist_ok=True)

In [28]:
azo_cis_2_path = data_path / 'azo_cis_2'
azo_cis_2_index = [106, 110]
azo_cis_2_pressure = [3.3, 30]
azo_cis_2_I_path = [azo_cis_2_path / f'azocis02_{i}_{i}_combined_I.dat' for i in azo_cis_2_index]
azo_cis_2_Qxy_path = [azo_cis_2_path / f'azocis02_{i}_{i}_combined_Qxy.dat' for i in azo_cis_2_index]
azo_cis_2_Qz_path = [azo_cis_2_path / f'azocis02_{i}_{i}_combined_Qz.dat' for i in azo_cis_2_index]
azo_cis_2_I = [np.loadtxt(path) for path in azo_cis_2_I_path]
azo_cis_2_Qxy = [np.loadtxt(path) for path in azo_cis_2_Qxy_path]
azo_cis_2_Qz = [np.loadtxt(path) for path in azo_cis_2_Qz_path]
azo_cis_2_I -= water_I[0]
azo_cis_2_save_path = plot_path / 'azo_cis_2'
azo_cis_2_save_path.mkdir(parents=True, exist_ok=True)

In [29]:
azo_cis_3_path = data_path / 'azo_cis_3'
azo_cis_3_index = [115, 119]
azo_cis_3_pressure = [0.1, 30]
azo_cis_3_I_path = [azo_cis_3_path / f'azocis03_{i}_{i}_combined_I.dat' for i in azo_cis_3_index]
azo_cis_3_Qxy_path = [azo_cis_3_path / f'azocis03_{i}_{i}_combined_Qxy.dat' for i in azo_cis_3_index]
azo_cis_3_Qz_path = [azo_cis_3_path / f'azocis03_{i}_{i}_combined_Qz.dat' for i in azo_cis_3_index]
azo_cis_3_I = [np.loadtxt(path) for path in azo_cis_3_I_path]
azo_cis_3_Qxy = [np.loadtxt(path) for path in azo_cis_3_Qxy_path]
azo_cis_3_Qz = [np.loadtxt(path) for path in azo_cis_3_Qz_path]
azo_cis_3_I -= water_I[0]
azo_cis_3_save_path = plot_path / 'azo_cis_3'
azo_cis_3_save_path.mkdir(parents=True, exist_ok=True)

In [30]:
dopc_path = data_path / 'dopc'
dopc_index = [16, 12]
dopc_pressure = [0.1, 10]
dopc_I_path = [dopc_path / f'dopc_{i}_{i}_combined_I.dat' for i in dopc_index]
dopc_Qxy_path = [dopc_path / f'dopc_{i}_{i}_combined_Qxy.dat' for i in dopc_index]
dopc_Qz_path = [dopc_path / f'dopc_{i}_{i}_combined_Qz.dat' for i in dopc_index]
dopc_I = [np.loadtxt(path) for path in dopc_I_path]
dopc_Qxy = [np.loadtxt(path) for path in dopc_Qxy_path]
dopc_Qz = [np.loadtxt(path) for path in dopc_Qz_path]
dopc_I -= water_I[0]
dopc_save_path = plot_path / 'dopc'
dopc_save_path.mkdir(parents=True, exist_ok=True)

In [31]:
red_azo_path = data_path / 'red_azo'
red_azo_index = [128, 132, 136, 140]
red_azo_pressure = [0.1, 10, 20, 30]
red_azo_I_path = [red_azo_path / f'redazo_{i}_{i}_combined_I.dat' for i in red_azo_index]
red_azo_Qxy_path = [red_azo_path / f'redazo_{i}_{i}_combined_Qxy.dat' for i in red_azo_index]
red_azo_Qz_path = [red_azo_path / f'redazo_{i}_{i}_combined_Qz.dat' for i in red_azo_index]
red_azo_I = [np.loadtxt(path) for path in red_azo_I_path]
red_azo_Qxy = [np.loadtxt(path) for path in red_azo_Qxy_path]
red_azo_Qz = [np.loadtxt(path) for path in red_azo_Qz_path]
red_azo_I -= water_I[0]
red_azo_save_path = plot_path / 'red_azo'
red_azo_save_path.mkdir(parents=True, exist_ok=True)

Plot data

In [32]:
from xrd_anlyutils.plot import mpl

In [33]:
for I, Q_xy, Q_z, index, pressure in zip(azo_cis_1_I, azo_cis_1_Qxy, azo_cis_1_Qz, azo_cis_1_index, azo_cis_1_pressure):
    mpl.plot_images([I],
                nrows=1,
                ncols=1,
                titles=[f'azo_cis_1 scan {index}: {pressure} mN/m'],
                origin='lower',
                vmin=0,
                vmax=30,
                figsize=(6, 6),
                extent=(Q_xy[0], Q_xy[-1], Q_z[0], Q_z[-1]),
                xlabel='Q_xy (A$^{-1}$)',
                ylabel='Q_z (A$^{-1}$)',
                save_path=azo_cis_1_save_path / f'azo_cis_1_scan_{index}_pressure_{pressure}mN_m.png',
                )

In [34]:
for I, Q_xy, Q_z, index, pressure in zip(azo_cis_2_I, azo_cis_2_Qxy, azo_cis_2_Qz, azo_cis_2_index, azo_cis_2_pressure):
    mpl.plot_images([I],
                nrows=1,
                ncols=1,
                titles=[f'azo_cis_2 scan {index}: {pressure} mN/m'],
                origin='lower',
                vmin=0,
                vmax=30,
                figsize=(6, 6),
                extent=(Q_xy[0], Q_xy[-1], Q_z[0], Q_z[-1]),
                xlabel='Q_xy (A$^{-1}$)',
                ylabel='Q_z (A$^{-1}$)',
                save_path=azo_cis_2_save_path / f'azo_cis_2_scan_{index}_pressure_{pressure}mN_m.png',
                )

In [35]:
for I, Q_xy, Q_z, index, pressure in zip(azo_cis_3_I, azo_cis_3_Qxy, azo_cis_3_Qz, azo_cis_3_index, azo_cis_3_pressure):
    mpl.plot_images([I],
                nrows=1,
                ncols=1,
                titles=[f'azo_cis_3 scan {index}: {pressure} mN/m'],
                origin='lower',
                vmin=0,
                vmax=30,
                figsize=(6, 6),
                extent=(Q_xy[0], Q_xy[-1], Q_z[0], Q_z[-1]),
                xlabel='Q_xy (A$^{-1}$)',
                ylabel='Q_z (A$^{-1}$)',
                save_path=azo_cis_3_save_path / f'azo_cis_3_scan_{index}_pressure_{pressure}mN_m.png',
                )

In [36]:
for I, Q_xy, Q_z, index, pressure in zip(dopc_I, dopc_Qxy, dopc_Qz, dopc_index, dopc_pressure):
    mpl.plot_images([I],
                nrows=1,
                ncols=1,
                titles=[f'dopc scan {index}: {pressure} mN/m'],
                origin='lower',
                vmin=0,
                vmax=30,
                figsize=(6, 6),
                extent=(Q_xy[0], Q_xy[-1], Q_z[0], Q_z[-1]),
                xlabel='Q_xy (A$^{-1}$)',
                ylabel='Q_z (A$^{-1}$)',
                save_path=dopc_save_path / f'dopc_scan_{index}_pressure_{pressure}mN_m.png',
                )

In [37]:
for I, Q_xy, Q_z, index, pressure in zip(red_azo_I, red_azo_Qxy, red_azo_Qz, red_azo_index, red_azo_pressure):
    mpl.plot_images([I],
                nrows=1,
                ncols=1,
                titles=[f'red_azo scan {index}: {pressure} mN/m'],
                origin='lower',
                vmin=0,
                vmax=30,
                figsize=(6, 6),
                extent=(Q_xy[0], Q_xy[-1], Q_z[0], Q_z[-1]),
                xlabel='Q_xy (A$^{-1}$)',
                ylabel='Q_z (A$^{-1}$)',
                save_path=red_azo_save_path / f'red_azo_scan_{index}_pressure_{pressure}mN_m.png',
                )

Process data

In [38]:
from xrd_anlyutils.math import roi

In [39]:
bbox = [Q_xy[0], Q_xy[-1], 0, 0.25]

In [40]:
azo_cis_1_roi = [roi.get_sum(bbox, I, Q_xy, Q_z, axis=0) for I, Q_xy, Q_z in zip(azo_cis_1_I, azo_cis_1_Qxy, azo_cis_1_Qz)]
azo_cis_2_roi = [roi.get_sum(bbox, I, Q_xy, Q_z, axis=0) for I, Q_xy, Q_z in zip(azo_cis_2_I, azo_cis_2_Qxy, azo_cis_2_Qz)]
azo_cis_3_roi = [roi.get_sum(bbox, I, Q_xy, Q_z, axis=0) for I, Q_xy, Q_z in zip(azo_cis_3_I, azo_cis_3_Qxy, azo_cis_3_Qz)]
dopc_roi = [roi.get_sum(bbox, I, Q_xy, Q_z, axis=0) for I, Q_xy, Q_z in zip(dopc_I, dopc_Qxy, dopc_Qz)]
red_azo_roi = [roi.get_sum(bbox, I, Q_xy, Q_z, axis=0) for I, Q_xy, Q_z in zip(red_azo_I, red_azo_Qxy, red_azo_Qz)]

In [41]:
mpl.plot_saxs(
    [Q_xy]*len(azo_cis_1_roi),
    azo_cis_1_roi,
    scatter=True,
    log_scale=False,
    xlabel='Q_xy (A$^{-1}$)',
    ylabel='I (a.u.)',
    save_path=azo_cis_1_save_path / 'azo_cis_1_roi.png',
    title=f'azo_cis_1 roi',
    labels=[f'scan {index}: {pressure} mN/m' for index, pressure in zip(azo_cis_1_index, azo_cis_1_pressure)],
    )


In [42]:
mpl.plot_saxs(
    [Q_xy]*len(azo_cis_2_roi),
    azo_cis_2_roi,
    scatter=True,
    log_scale=False,
    xlabel='Q_xy (A$^{-1}$)',
    ylabel='I (a.u.)',
    save_path=azo_cis_2_save_path / 'azo_cis_2_roi.png',
    title=f'azo_cis_2 roi',
    labels=[f'scan {index}: {pressure} mN/m' for index, pressure in zip(azo_cis_2_index, azo_cis_2_pressure)],
    )

In [43]:
mpl.plot_saxs(
    [Q_xy]*len(azo_cis_3_roi),
    azo_cis_3_roi,
    scatter=True,
    log_scale=False,
    xlabel='Q_xy (A$^{-1}$)',
    ylabel='I (a.u.)',
    save_path=azo_cis_3_save_path / 'azo_cis_3_roi.png',
    title=f'azo_cis_3 roi',
    labels=[f'scan {index}: {pressure} mN/m' for index, pressure in zip(azo_cis_3_index, azo_cis_3_pressure)],
    )

In [44]:
mpl.plot_saxs(
    [Q_xy]*len(dopc_roi),
    dopc_roi,
    scatter=True,
    log_scale=False,
    xlabel='Q_xy (A$^{-1}$)',
    ylabel='I (a.u.)',
    save_path=dopc_save_path / 'dopc_roi.png',
    title=f'dopc roi',
    labels=[f'scan {index}: {pressure} mN/m' for index, pressure in zip(dopc_index, dopc_pressure)],
    )

In [45]:
mpl.plot_saxs(
    [Q_xy]*len(red_azo_roi),
    red_azo_roi,
    scatter=True,
    log_scale=False,
    xlabel='Q_xy (A$^{-1}$)',
    ylabel='I (a.u.)',
    save_path=red_azo_save_path / 'red_azo_roi.png',
    title=f'red_azo roi',
    labels=[f'scan {index}: {pressure} mN/m' for index, pressure in zip(red_azo_index, red_azo_pressure)],
    )

Fit lorentzian